In [3]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
from os.path import join
import pandas as pd
import joblib
import sasc.config
import numpy as np
from collections import defaultdict
import pandas as pd
# import story_helper
from sasc.modules.fmri_module import convert_module_num_to_voxel_num, add_stability_score
from sasc.config import FMRI_DIR, STORIES_DIR
from sasc import config

# Read all the info from stories into a single pickle file

In [4]:
# load stuff
# double check all of these, intro paragraph may be the same...
output_file = join(sasc.config.RESULTS_DIR,
                   'processed', "pilot4_story_data.pkl")
story_mapping = {
    'default/uts01___may9___seed=5_top1': 'deeptune-story19.npy',
    'default/uts01___may9___seed=2_top2': 'deeptune-story20.npy',

    'interactions/uts01___may9___seed=3_top1': 'deeptune-story21.npy',
    'interactions/uts01___may9___seed=6_top2': 'deeptune-story22.npy',
}

# custom timings because the story had to be slowed down
story_timings_custom = {
    'default/uts01___may9___seed=5_top1': 'UTS01_deeptune19.report',
    'default/uts01___may9___seed=2_top2': 'UTS01_deeptune20.report',

    'interactions/uts01___may9___seed=3_top1': 'UTS01_deeptune21.report',
    'interactions/uts01___may9___seed=6_top2': 'UTS01_deeptune22.report',
}

In [5]:
# add header line to report files
header_line = "time misc word misc"
for val in story_timings_custom.values():
    original_file = join(config.PILOT_STORY_DATA_DIR,
                         '20240509', val)
    new_file = join(config.PILOT_STORY_DATA_DIR,
                    '20240509', val.replace('.report', '_processed.txt'))
    with open(original_file, 'r') as file:
        s = file.read()
    with open(new_file, 'w') as file:
        file.write(header_line + '\n' + s.replace('"', ''))

In [11]:
# cluster_neighbors = joblib.load(join(FMRI_DIR, "voxel_neighbors_and_pcs", "cluster_neighbors_v1.pkl"))
perfs = joblib.load(join(sasc.config.FMRI_DIR, 'sasc', 'rj_models',
                    'opt_model', 'new_setup_performance.jbl'))

# add keys
stories_data_dict = defaultdict(list)
for story_idx, story_name in enumerate(story_mapping.keys()):
    # add scalar story descriptions
    stories_data_dict["story_name_original"].append(story_name)
    stories_data_dict["story_setting"].append(story_name.split("/")[0])
    stories_data_dict["story_name_new"].append(story_mapping[story_name])
    stories_data_dict["story_text"].append(
        open(join(STORIES_DIR, story_name, "story.txt"), "r").read()
    )
    prompts_paragraphs = joblib.load(
        join(STORIES_DIR, story_name, "prompts_paragraphs.pkl")
    )

    # add paragraph-level descriptions
    timings_file = join(config.PILOT_STORY_DATA_DIR, '20240509',
                        story_timings_custom[story_name]).replace('.report', '_processed.txt')
    print(timings_file)
    # , quotechar=None, quoting=3)
    timings = pd.read_csv(timings_file, sep=' ')
    timings = timings[timings.misc == 'word']
    timings['time_running'] = timings['time']
    timings = timings.reset_index()

    timings_to_save = timings.copy()
    # timings_to_save['time_running'] = timings_to_save['time_running'] - 10
    timings_to_save['timing'] = np.diff(
        timings_to_save['time_running']).tolist() + [0]

    timings_to_save.to_csv(
        join(STORIES_DIR, story_name, "timings_processed_slowed.csv")
    )
    # display(timings)
    # timings['time_running'] = timings['time_running'] + 10

    stories_data_dict["timing"].append(
        # pd.read_csv(join(STORIES_DIR, story_name, "timings_processed.csv"))
        timings
    )
    stories_data_dict["prompts"].append(prompts_paragraphs["prompts"])
    stories_data_dict["paragraphs"].append(prompts_paragraphs["paragraphs"])

    # add paragraph-level metadata
    # rows
    # rows = pd.read_csv(join(STORIES_DIR, story_name, "rows.csv"))
    story_metadata_per_paragraph = pd.read_pickle(
        join(STORIES_DIR, story_name, "rows.pkl"))
    story_metadata_per_paragraph["voxel_num"] = story_metadata_per_paragraph.apply(
        lambda row: convert_module_num_to_voxel_num(
            row["module_num"], row["subject"]),
        axis=1,
    )
    story_metadata_per_paragraph["stability_score"] = story_metadata_per_paragraph.apply(
        lambda row: add_stability_score(row["module_num"], row["subject"]),
        axis=1,
    )
    story_metadata_per_paragraph = story_metadata_per_paragraph[
        [
            "expl",
            "module_num",
            "top_explanation_init_strs",
            "subject",
            "fmri_test_corr",
            "stability_score",
            # "top_score_synthetic",
            "top_score_normalized",
            "roi_anat",
            "roi_func",
            "voxel_num",
        ]
    ]
    story_metadata_per_paragraph['test_corr_new'] = story_metadata_per_paragraph['voxel_num'].apply(
        lambda x: perfs[x])
    # rows['cluster_nums'] = rows['voxel_num'].map(cluster_neighbors)
    stories_data_dict["rows"].append(story_metadata_per_paragraph)

    if "interactions" in list(story_mapping.keys())[story_idx]:
        rows1 = pd.read_pickle(join(STORIES_DIR, story_name, "rows1.pkl"))
        rows2 = pd.read_pickle(join(STORIES_DIR, story_name, "rows2.pkl"))
        rows1["voxel_num"] = rows1.apply(
            lambda row: convert_module_num_to_voxel_num(
                row["module_num"], row["subject"]
            ),
            axis=1,
        )
        rows2["voxel_num"] = rows2.apply(
            lambda row: convert_module_num_to_voxel_num(
                row["module_num"], row["subject"]
            ),
            axis=1,
        )
        stories_data_dict['voxel_num1'].append(rows1['voxel_num'])
        stories_data_dict['voxel_num2'].append(rows2['voxel_num'])
        stories_data_dict['expl1'].append(rows1['expl'])
        stories_data_dict['expl2'].append(rows2['expl'])
    else:
        stories_data_dict['voxel_num1'].append([])
        stories_data_dict['voxel_num2'].append([])
        stories_data_dict['expl1'].append([])
        stories_data_dict['expl2'].append([])


joblib.dump(stories_data_dict, output_file)

/home/chansingh/mntv1/deep-fMRI/brain_tune/story_data/20240509/UTS01_deeptune19_processed.txt
/home/chansingh/mntv1/deep-fMRI/brain_tune/story_data/20240509/UTS01_deeptune20_processed.txt
/home/chansingh/mntv1/deep-fMRI/brain_tune/story_data/20240509/UTS01_deeptune21_processed.txt
/home/chansingh/mntv1/deep-fMRI/brain_tune/story_data/20240509/UTS01_deeptune22_processed.txt


['/home/chansingh/automated-explanations/results/processed/pilot4_story_data.pkl']

In [9]:
# display full df
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
# display(stories_data_dict['timing'][0])

In [10]:
timings['word'].values[359]

'Margot'